In [1]:
with open(r'../Netcrawler/fitbit_extracted.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

In [2]:
text_data=text_data.strip('[]').split(', ')
text_data

# Lowercase all the paragraphs in the list
text_data = [paragraph.lower() for paragraph in text_data]

In [3]:
text_data

['"\\non the first day of each month',
 'your fitbit sleep profile reveals which animal represents your most recent sleep habits. fun fact - the animal with the highest proportion of women is the tortoise',
 'who tends to fall asleep more slowly and get a reasonable amount of sleep overall',
 "while the hedgehog has the highest proportion of men.\\n what’s your sleep animal for the last month? \\n\\n\\nmine is consistently bear.\\xa0🐻\\n\\nrieko | n california usa mbg pe \\n\\n\\n\\nis this only with the premium membership? i don't see anything.\\n\\n\\n@shewolf792\\xa0yes",
 'the sleep animal is a premium feature. also',
 'it is only available with certain fitbit models.\\xa0\\n\\namanda | wyoming',
 'usa\\n\\n\\n\\nthanks for info. it\'s appreciated.\\xa0\\n\\n\\nmine was the giraffe (first month using the fitbit).\\n"',
 "'\\nthe personalized sleep profile with fitbit premium goes beyond nightly tracking to analyze your sleep over a month-long period. learn more in part 1\\xa0and pa

In [4]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Step 1: Text Cleaning
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)#

    # Remove newline
    text = re.sub(r'\\n', '', text)
    
    # Remove special characters except spaces
    text = re.sub(r'[^\w\s]', '', text)

    # Remove all numbers
    text = re.sub(r'\d', '', text)
    
    # Remove extra spaces and trim the text
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


cleaned_data = [clean_text(text) for text in text_data]
# delete xa0
cleaned_data = [text.replace('xa0', ' ') for text in cleaned_data]


In [6]:
# Step 2: Tokenization
nltk.download('punkt')  # Download the NLTK tokenizer data if not already downloaded

def tokenize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    return words

tokenized_data = [tokenize_text(text) for text in cleaned_data]


[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [7]:
# Step 3: Stopword Removal
# open stopword file (source: https://countwordsfree.com/stopwords)
with open(r'stop_words_english.txt', 'r', encoding='utf-8') as file:
    stopwordlist = file.read()

# Split the string into a list of stop words using line breaks as the separator
stopwordslist = stopwordlist.split('\n')

# Remove any empty strings from the list
stopwordslist = [word for word in stopwordlist if word.strip()]


stopwordlistpers = ["night", "time", "minute", "ive", "one", "got", "dont", "lot" 
                , "getting", "good", "much", "first", "give", "thank", "might"
                , "really", "im", "fitbit", "te", "would", "else", "hour", "cant" 
                , "sleep", "want", "back", "around", "month", "usaversa", "luxe", "aria"
                , "fit bit", "maryland", "flwant", ]

def remove_stopwords(tokens, stop_words):
    return [word for word in tokens if word not in stop_words]

# Remove common stopwords
filtered_data = [remove_stopwords(tokens, stopwordlist) for tokens in tokenized_data]

# Remove personalized stopwords
filtered_data = [remove_stopwords(tokens, stopwordlistpers) for tokens in filtered_data]

# Remove blanco
filtered_data = [sublist for sublist in filtered_data if sublist]


In [8]:
# Step 4: Lemmatization
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

lemmatized_data = [lemmatize_tokens(tokens) for tokens in filtered_data]


In [9]:
# Step 5: Rejoin Tokens into Text
preprocessed_data = [' '.join(tokens) for tokens in lemmatized_data]

In [10]:
%store preprocessed_data
%store lemmatized_data
%store cleaned_data

Stored 'preprocessed_data' (list)
Stored 'lemmatized_data' (list)
Stored 'cleaned_data' (list)
